# Problème Kowalik *(Enzyme problem)*

Source: [à déterminer]

$ n = 4,\ m = 11$

$\left\{  
\begin{array}{l l l} 
\underset{x}{\min} \|F(x)\|^2 = \sum_{k=1}^{m} F_k(x)^2\\ 
\text{s.c.}\\
x \in [a,b] \\
\end{array} \right.$

où $ F_k(x) = V_k - \dfrac{x_1\left( y_k^2+x_2y_k \right)}{y_k^2+x_3y_k+x_4} $

# Résolution avec ENLSIP-Julia

In [1]:
using ForwardDiff, Distributions

In [21]:
include("../src/enlsip_functions_castor2.jl")

enlsip

In [3]:
n = 4
m = 11
nb_eq = 0
nb_constraints = n * 2

8

In [4]:
# Datapoints


V = vec([.1957 .1947 .1735 .1600 .0844 .0627 .0456 .0342 .0323 .0235 .0246])
y = vec([0 10 20 30 40 50 60 70 80 90 100]);

In [12]:
# Résidus

function r_k(x::Vector, y_k::Number, V_k::Number)
    return V_k - (x[1]*(y_k^2 + x[2]*y_k)) / (y_k^2+x[3]*y_k+x[4])
end

function r(x::Vector) 
    return [r_k(x,y[k],V[k]) for k = 1:m]
end

resKowalik = ResidualsEval(0)

function (resKowalik::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)
    
    if resKowalik.ctrl == 1
        rx[:] = r(x)
    elseif resKowalik.ctrl == 2
        J[:] = ForwardDiff.jacobian(r,x)
    end
    return
end

In [19]:
# Contraintes 

function c(x::Vector)
    return [x[1] - 0.1928; 1 - x[1];
            x[2] - 0.1916; 1 - x[2];
            x[3] - 0.1234; 1 - x[3];
            x[4] - 0.1362; 1 - x[4]]
end

consKowalik = ConstraintsEval(0)

function (consKowalik::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)
    if consKowalik.ctrl == 1
        cx[:] = [x[1] - 0.1928; 1 - x[1];
            x[2] - 0.1916; 1 - x[2];
            x[3] - 0.1234; 1 - x[3];
            x[4] - 0.1362; 1 - x[4]]
    elseif consKowalik == 2
        A[:] = ForwardDiff.jacobian(c,x)
    end
    return
end

In [7]:
generate_starting_point = (lb::Vector,ub::Vector) -> [rand(Uniform(a,b)) for (a,b) in zip(lb,ub)]

#59 (generic function with 1 method)

In [59]:
lb =  [0.1928, 0.1916, 0.1234, 0.1362]
ub = [1.0, 1.0, 1.0, 1.0]

x0 = generate_starting_point(lb,ub) # [0.19281; 0.19165; 0.12340; 13620]

e = eps(Float64)
se = sqrt(e)
enlsipKowalik = enlsip(x0,resKowalik,consKowalik,n,m,nb_eq,nb_constraints,ε_abs = e, ε_rel = se, ε_x = se, ε_c = se)


****************************************
*                                      *
*          ENLSIP-JULIA-0.4.0          *
*                                      *
****************************************

Starting point : [0.40300531699423275, 0.5005776076085529, 0.795691841092174, 0.31016039251919664]

Starting point :
    4.030053e-01   5.005776e-01   7.956918e-01   3.101604e-01 

Number of equality constraints   : 0
Number of inequality constraints : 8
Constraints internal scaling     : false


Iteration steps information

iter     objective    cx_sum   reduction     ||p||   dimA  dimJ2     α     conv. speed   max weight   working set
   0  1.080363e+00  0.00e+00   6.42e-03   2.085e+02   0     4   3.07e-03    0.00e+00     1.00e-01     -
   1  1.073942e+00  8.13e-03   7.87e-03   1.169e+02   0     3   3.80e-03    9.97e-01     1.00e-01    (6)
   2  1.066075e+00  4.76e-02   2.32e-02   6.052e+00   0     2   1.14e-02    9.96e-01     1.00e-01    (6,8)
   3  1.042914e+00  4.76e-02   6.74e

ENLSIP(-6, [0.0838585747561788, 1.1964809896760606, 1.0901888318577428, 1.198738562244937], 0.07862502288875073)

In [57]:
using JuMP, Ipopt

In [62]:
lb =  [0.1928, 0.1916, 0.1234, 0.1362]
ub = [1.0, 1.0, 1.0, 1.0]

x0 = generate_starting_point(lb,ub)

model = Model(Ipopt.Optimizer)

@variable(model, lb[i] <= x[i=1:n] <= ub[i], start=x0[i])

@NLobjective(model, Min, sum((V[k] - (x[1]*(y[k]^2 + x[2]*y[k])) / (y[k]^2+x[3]*y[k]+x[4]))^2 for k=1:m))

optimize!(model)

value.(model[:x])

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  6.4355180e+00 0.00e+00 1.60e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

4-element Vector{Float64}:
 0.19279999121139316
 0.19160042633489713
 0.9999995702595886
 0.9999568035746623